In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import copy

In [2]:
def show_v_table_small(v_table, env):
    for i in range(env.reward.shape[0]):        
        print("+----------"*env.reward.shape[1])
        print("|", end="")
        for j in range(env.reward.shape[1]):
            print("{0:8.2f}  |".format(v_table[i,j]),end="")
        print()
    print("+----------"*env.reward.shape[1])

# V table 그리기    
def show_v_table(v_table, env):    
    for i in range(env.reward.shape[0]):        
        print("+-----------------"*env.reward.shape[1],end="")
        print("+")
        for k in range(3):
            print("|",end="")
            for j in range(env.reward.shape[1]):
                if k==0:
                    print("                 |",end="")
                if k==1:
                        print("   {0:8.2f}      |".format(v_table[i,j]),end="")
                if k==2:
                    print("                 |",end="")
            print()
    print("+-----------------"*env.reward.shape[1],end="")
    print("+")
    
# Q table 그리기
def show_q_table(q_table,env):
    for i in range(env.reward.shape[0]):
        print("+-----------------"*env.reward.shape[1],end="")
        print("+")
        for k in range(3):
            print("|",end="")
            for j in range(env.reward.shape[1]):
                if k==0:
                    print("{0:10.2f}       |".format(q_table[i,j,0]),end="")
                if k==1:
                    print("{0:6.2f}    {1:6.2f} |".format(q_table[i,j,3],q_table[i,j,1]),end="")
                if k==2:
                    print("{0:10.2f}       |".format(q_table[i,j,2]),end="")
            print()
    print("+-----------------"*env.reward.shape[1],end="")
    print("+")
    

# 정책 policy 화살표로 그리기
def show_q_table_arrow(q_table,env):
    for i in range(env.reward.shape[0]):        
        print("+-----------------"*env.reward.shape[1],end="")
        print("+")
        for k in range(3):
            print("|",end="")
            for j in range(env.reward.shape[1]):
                if k==0:
                    if np.max(q[i,j,:]) == q[i,j,0]:
                        print("        ↑       |",end="")
                    else:
                        print("                 |",end="")
                if k==1:                    
                    if np.max(q[i,j,:]) == q[i,j,1] and np.max(q[i,j,:]) == q[i,j,3]:
                        print("      ←  →     |",end="")
                    elif np.max(q[i,j,:]) == q[i,j,1]:
                        print("          →     |",end="")
                    elif np.max(q[i,j,:]) == q[i,j,3]:
                        print("      ←         |",end="")
                    else:
                        print("                 |",end="")
                if k==2:
                    if np.max(q[i,j,:]) == q[i,j,2]:
                        print("        ↓       |",end="")
                    else:
                        print("                 |",end="")
            print()
    print("+-----------------"*env.reward.shape[1],end="")
    print("+")    
    
# 정책 policy 화살표로 그리기
def show_policy_small(policy,env):
    for i in range(env.reward.shape[0]):        
        print("+----------"*env.reward.shape[1],end="")
        print("+")
        print("|", end="")
        for j in range(env.reward.shape[1]):
            if env.reward_list1[i][j] == "road":
                if policy[i,j] == 0:
                    print("   ↑     |",end="")
                elif policy[i,j] == 1:
                    print("   →     |",end="")
                elif policy[i,j] == 2:
                    print("   ↓     |",end="")
                elif policy[i,j] == 3:
                    print("   ←     |",end="")
            else:
                print("          |",end="")
        print()
    print("+----------"*env.reward.shape[1],end="")
    print("+")
    
# 정책 policy 화살표로 그리기
def show_policy(policy,env):
    for i in range(env.reward.shape[0]):        
        print("+-----------------"*env.reward.shape[1],end="")
        print("+")
        for k in range(3):
            print("|",end="")
            for j in range(env.reward.shape[1]):
                if k==0:
                    print("                 |",end="")
                if k==1:
                    if policy[i,j] == 0:
                        print("      ↑         |",end="")
                    elif policy[i,j] == 1:
                        print("      →         |",end="")
                    elif policy[i,j] == 2:
                        print("      ↓         |",end="")
                    elif policy[i,j] == 3:
                        print("      ←         |",end="")
                if k==2:
                    print("                 |",end="")
            print()
    print("+-----------------"*env.reward.shape[1],end="")
    print("+")

In [3]:
class Agent():
    
    # 1. 행동에 따른 에이전트의 좌표 이동(위, 오른쪽, 아래, 왼쪽) 
    action = np.array([[-1,0],[0,1],[1,0],[0,-1]])
    
    # 2. 각 행동별 선택확률
    select_action_pr = np.array([0.25,0.25,0.25,0.25])
    
    # 3. 에이전트의 초기 위치 저장
    def __init__(self):
        self.pos = (0,0)
    
    # 4. 에이전트의 위치 저장
    def set_pos(self,position):
        self.pos = position
        return self.pos
    
    # 5. 에이전트의 위치 불러오기
    def get_pos(self):
        return self.pos

In [4]:
class Environment():
    
    # 1. 미로밖(절벽), 길, 목적지와 보상 설정
    cliff = -3
    road = -1
    goal = 1
    
    # 2. 목적지 좌표 설정
    goal_position = [2,2]
    
    # 3. 보상 리스트 숫자
    reward_list = [[road,road,road],
                   [road,road,road],
                   [road,road,goal]]
    
    # 4. 보상 리스트 문자
    reward_list1 = [["road","road","road"],
                    ["road","road","road"],
                    ["road","road","goal"]]
    
    # 5. 보상 리스트를 array로 설정
    def __init__(self):
        self.reward = np.asarray(self.reward_list)    

    # 6. 선택된 에이전트의 행동 결과 반환 (미로밖일 경우 이전 좌표로 다시 복귀)
    def move(self, agent, action):
        
        done = False
        
        # 6.1 행동에 따른 좌표 구하기
        new_pos = agent.pos + agent.action[action]
        
        # 6.2 현재좌표가 목적지 인지확인
        if self.reward_list1[agent.pos[0]][agent.pos[1]] == "goal":
            reward = self.goal
            observation = agent.set_pos(agent.pos)
            done = True
        # 6.3 이동 후 좌표가 미로 밖인 확인    
        elif new_pos[0] < 0 or new_pos[0] >= self.reward.shape[0] or new_pos[1] < 0 or new_pos[1] >= self.reward.shape[1]:
            reward = self.cliff
            observation = agent.set_pos(agent.pos)
            done = True
        # 6.4 이동 후 좌표가 길이라면
        else:
            observation = agent.set_pos(new_pos)
            reward = self.reward[observation[0],observation[1]]
            
        return observation, reward, done

In [6]:
def generate_episode(env, agent, first_visit):
    gamma = 0.09

    episode = []
    visit = np.zeros((env.reward.shape[0],env.reward.shape[1]))
    #무작위 위치 설정
    i = np.random.randint(0, env.reward.shape[0])
    j = np.random.randint(0, env.reward.shape[1])
    agent.set_pos([i, j])

    G = 0

    step = 0
    max_step = 100

    #에피소드 생성
    for k in range(max_step):
        pos = agent.get_pos()
        action = np.random.randint(0, len(agent.action))
        observation, reward, done = env.move(agent, action)

        if first_visit:
            #에피소드에 방문한 적 없을 경우
            if visit[pos[0],pos[1]] == 0:
                G += gamma**(step) * reward
                #방문 표시
                visit[pos[0],pos[1]] = 1
                step +=1
                #방문 이력 저장
                episode.append((pos, action, reward))
        else:
            #everyvisit MC
            G += gamma**(step) * reward
            step +=1
            episode.append((pos, action, reward))
        if done:
            break
    return i, j, G, episode

In [9]:
np.random.seed(0)

#환경, 에이전트 초기화
env = Environment()
agent = Agent()

#임의의 상태 가치 함수 V
v_table = np.zeros((env.reward.shape[0], env.reward.shape[1]))
#상태별로 에피소드 출발 횟수를 저장하는 테이블
v_start = np.zeros((env.reward.shape[0], env.reward.shape[1]))
#상태별로 에피소드 종료 횟수를 저장하는 테이블
v_success = np.zeros((env.reward.shape[0], env.reward.shape[1]))
#3x3 empty list Return_s
Return_s = [[[] for _ in range(env.reward.shape[1])]
            for _ in range(env.reward.shape[0])]
#최대 에피소드 수 지정
max_episode = 100000

first_visit = True
if first_visit:
    print("start first visit MC")
else:
    print("start every visit MC")
print()

for epi in tqdm(range(max_episode)):
    i, j, G, episode = generate_episode(env, agent, first_visit)
    # #수익 G를 Return_s에 추가
    # Return_s[i][j].append(G)
    # #에피소드 발생 횟수 계산
    # episode_count = len(Return_s[i][j])
    # #상태별 발생한 수익의 총합 계산
    # total_G = np.sum(Return_s[i][j])
    # #상태별 발생한 수익의 평균 계산
    # v_table[i, j] = total_G / episode_count
    # #도착지점에 도착했는지 체크
    # if episode[-1][2] == 1:
    #     v_success[i, j] += 1
    #incremental mean
    v_visit = v_start[i, j] + 1
    v_table[i, j] += 1/v_visit * (G - v_table[i, j])

#에피소드 출발 횟수 저장
for i in range(env.reward.shape[0]):
    for j in range(env.reward.shape[1]):
        v_start[i, j] = len(Return_s[i][j])

print("V(s)")
show_v_table(np.round(v_table, 2), env)
# print("V(s)_start")
# show_v_table(np.round(v_start, 2), env)
# print("V(s)_success")
# show_v_table(np.round(v_success/v_start, 2), env)


start first visit MC



100%|██████████| 100000/100000 [00:03<00:00, 28108.84it/s]

V(s)
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -3.00      |      -1.27      |      -1.27      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -1.27      |      -1.11      |      -1.09      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
|                 |                 |                 |
|      -3.00      |      -1.08      |       1.00      |
|                 |                 |                 |
+-----------------+-----------------+-----------------+
